In [11]:
import scvi
from scipy.sparse import vstack, csr_matrix
import scanpy as sc
import anndata as ad

In [2]:
adata_path = '/home/icb/jonas.flor/gastrulation_atlas/scvi/training/10k/all_genes'
step = 1000

In [4]:
adata = sc.read(f'{adata_path}/unintegrated_adata.h5ad', backed='r')
vae = scvi.model.SCVI.load(dir_path=adata_path, adata=adata)

INFO     File /home/icb/jonas.flor/gastrulation_atlas/scvi/training/10k/all_genes/model.pt already downloaded      


/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/jonas.flor/mambaforge/envs/scvi/lib/python ...
  rank_zero_warn(
I0000 00:00:1698139962.221572  524087 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/icb/jonas.flor/mambaforge/envs/scvi/lib/python3.11/site-packages/scvi/data/fields/_dataframe_field.py:227: UserWarning: Category 5 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  new_mapping = _make_column_categorical(


In [8]:
X = vae.get_normalized_expression(indices = list(range(0,min(step, adata.n_obs)))).clip(lower=.1)
X[X<=0.1] = 0
X = csr_matrix(X)
X

<1000x24552 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
for i in range(min(step, adata.n_obs), adata.n_obs, step):
    Y = vae.get_normalized_expression(indices = range(i, min(i+step,adata.n_obs))).clip(lower=.1)
    Y[Y<=0.1] = 0
    Y = csr_matrix(Y)
    X = vstack([X, Y])

In [12]:
adata_int = ad.AnnData(X)
del X, Y

In [13]:
adata_int.obs = adata.obs
adata_int.var = adata.var
del adata

In [15]:
adata_int.obsm["X_emb"] = vae.get_latent_representation()
del vae